# Modelová úloha - hledání kam se dostaneme v grafu pokud můžeme-li ujít z vrcholu `0` maximálně `n` kroků
- Budeme uvažovat graf jako množinu vrcholů a hran. 
    - Vrcholy jsou čísla od `0` do `n-1`
    - Hrany jsou dvojice `(u, v)` takové, že `u` a `v` jsou vrcholy grafu
- Ujít z vrcholu `0` jeden krok znamená přejít do nějakého vrcholu, který je s `0` spojen hranou

### Vstupní data - list dvojic vrcholů = hrany
- vygenerujeme primitivní graf, který bude mít `n` vrcholů a `m` hran
    - pro každou hranu `(u, v)` vygenerujeme náhodně vybereme, zda v grafu bude nebo nebude
    - pravděpodobnost volíme tak, aby každý vrchol měl průměrně `d` hran

In [ ]:
import random
n = 10

def vygeneruj_graf(n, d = 3):
    V = [i for i in range(n)]
    prob = d / n
    E = []
    for v_i in range(n):
        for v_j in range(v_i + 1, n):
            if random.random() < prob:
                E.append((v_i, v_j))
    return V, E

V, E = vygeneruj_graf(n)

In [ ]:
print(V)
print(E)

### Ochutnávka knihovny networkx = vykreslení grafu

In [ ]:
# !pip install networkx

In [ ]:
# plot graph with vertices V and edges E
# showing vertices with numbers and connections as lines
import matplotlib.pyplot as plt
import networkx as nx

G = nx.Graph()
G.add_nodes_from(V)
G.add_edges_from(E)
pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G, pos)
nx.draw_networkx_edges(G, pos)
nx.draw_networkx_labels(G, pos)
plt.show()


### A pomocí GraphViz
Pěknější vizualizace grafu

- je třeba mít naistalovaný GraphViz `sudo apt-get install graphviz`
- bindingy do Pythonu `pip install graphviz`

In [ ]:
import graphviz

# Create a new graph
graph = graphviz.Graph()

# Add vertices to the graph
_ = [graph.node(str(vertex)) for vertex in V]

# Add edges to the graph
_ = [graph.edge(str(edge[0]), str(edge[1])) for edge in E]

# Render and display the graph
graph

## První návrh - jednoduchý Python, použití setů, listů a union
- Vytvoříme si množinu vrcholů, které už jsou dostupné
- V každém kroku zjistíme, které vrcholy můžeme dosáhnout z vrcholů, které už máme
    - tak, že projdeme všechny hrany ve kterých se vyskytuje alespoň jeden vrchol, který už máme
    - a druhý vrchol v dané hraně přidáme do množiny vrcholů, které už máme

In [ ]:
def reachable_in_n_steps(edges, n):
    reachable = set()
    reachable.add(0)
    for i in range(n):
        new_reachable = set()
        for v in reachable:
            for e in edges:
                if e[0] == v:
                    new_reachable.add(e[1])
                if e[1] == v:
                    new_reachable.add(e[0])
        reachable = reachable.union(new_reachable)
    return list(reachable)

In [ ]:
reachable_in_n_steps(E, 2)

## Vygenerujeme větší graf

In [ ]:
V, E = vygeneruj_graf(2000)

Jak dlouho to asi potrvá?

In [ ]:
%time _ = reachable_in_n_steps(E, 20)

In [ ]:
res1 = reachable_in_n_steps(E, 20)

## Profilování

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f reachable_in_n_steps reachable_in_n_steps(E, 20)

**Edge procházíme desítky miliónů krát... tohle ani rychlé být nemůže. Jak to můžeme zrychlit?**


## Optimalizace algoritmu

- Kolik krát vlastně musíme projít záznam s jednou hranou?
- Musíme kontrolovat znova sousedy vrcholu, který jsme už jednou kontrolovali?
- Pokud víme, že už jsou dostupné všechny vrcholy, jak je náročný výše uvedený algoritmus?
- V současné implementaci kontrolujeme pro každý vrchol, všechny hrany. Nebude jednodušší pro každou hranu zjistit, zda je některý z jejích vrcholů v množině vrcholů, které jsou dostupné?

Máme-li odpovězené tyto otázky, můžeme se pustit do lepšího algoritmu.

**Změny v novém návrhu algoritmu:**
- Budeme spravovat seznam nezpracovaných hran
    - pokud nějakou hranu projdeme, tak ji z tohoto seznamu odstraníme
- Budeme procházet jen ty vrcholy které jsme v minulém kroce přidali (a zároveň ještě nebyly zpracovány)
    - Pokud je takovýto seznam prázdný, tak už nemáme co zpracovávat a můžeme skončit.

**!!!Na začátku funkce budeme muset vyrobit kopii seznamu hran!!!**
- protože vstupní seznam budeme postupně mazat

In [ ]:
def reachable_in_n_steps_v2(edges_in: list, n: int):
    edges = edges_in.copy()  # kopie hran, abychom je mohli mazat
    reachable = set()
    reachable.add(0)  # začítáme ve vrcholu 0
    newly_reachable = reachable.copy()  # vrcholy, které jsme právě přidali

    for _ in range(n):  # počet kroků
        next_reachable = set()  # vrcholy, které budou dosažitelné z newly_reachable

        for e_idx, e in list(enumerate(edges))[::-1]:  # procházíme pozpátku pro snadnější mazání
            for v in newly_reachable:  # je některý z konců hrany dosažitelný?
                if e[0] == v:
                    next_reachable.add(e[1])
                    edges.pop(e_idx)
                    break  # pokud jsme našli hranu, nemusíme hledat dál
                if e[1] == v:
                    next_reachable.add(e[0])
                    edges.pop(e_idx)
                    break  # pokud jsme našli hranu, nemusíme hledat dál
        
        newly_reachable = next_reachable.difference(reachable)  # pouze nově přidané vrcholy
        reachable = reachable.union(next_reachable)  # přidáme nově dosažitelné vrcholy
        if not newly_reachable:
            break
        
    return list(reachable)

In [ ]:
%time res2 = reachable_in_n_steps_v2(E, 20)

Tohle už je aspoň nějaké zrychlení.

Raději ověříme, že počítáme stále to samé:

In [ ]:
import numpy as np
res2 = reachable_in_n_steps_v2(E, 20)
np.allclose(np.array(res1), np.array(res2))

In [ ]:
%lprun -f reachable_in_n_steps_v2 reachable_in_n_steps_v2(E, 5)

## Další algoritmická optimalizace: reprezentace grafu
- Co takhle reprezentovat hrany jako seznamy sousedů?
    - pro každý vrchol si budeme pamatovat seznam vrcholů, do kterých můžeme přejít
- Pak nemusíme dělat smyčku přes všechny hrany a vyhledávat, ale pouze přidáme do seznamu vrcholů seznam sousedů.

In [ ]:
def reachable_in_n_steps_v3(edges_in: list, n: int):
    n_vertices = max([max(e) for e in edges_in]) + 1  # počet vrcholů
    edges = [[] for _ in range(n_vertices)]  # seznam (zatím prázdný) sousedů pro každý vrchol

    for e in edges_in:  # pro každou hranu přidáme oboum vrcholům druhého souseda
        edges[e[0]].append(e[1])
        edges[e[1]].append(e[0])

    reachable = set()
    reachable.add(0)  # začítáme ve vrcholu 0
    newly_reachable = reachable.copy()  # vrcholy, které jsme právě přidali

    for _ in range(n):  # počet kroků
        next_reachable = set()  # vrcholy, které budou dosažitelné z newly_reachable
        for v in newly_reachable:
            [next_reachable.add(soused) for soused in edges[v]]  # přidáme sousedy

        newly_reachable = next_reachable.difference(reachable)  # pouze nově přidané vrcholy
        reachable = reachable.union(next_reachable)  # přidáme nově dosažitelné vrcholy
        if not newly_reachable:
            break

    return list(reachable)

In [ ]:
%timeit res2 = reachable_in_n_steps_v3(E, 20)

To je ještě o řád rychlejší. 

Zkontrolujeme, že děláme stále to samé.

In [ ]:
res2 = reachable_in_n_steps_v3(E, 20)
np.allclose(np.array(res1), np.array(res2))

In [ ]:
%lprun -f reachable_in_n_steps_v3 reachable_in_n_steps_v3(E, 20)

## Implementační optimalizace - použití NumPy a pole bool hodnot místo setů
- práce s listy a sety je pomalá
- zkusíme místo přidávání a odstraňování prvků do setů a seznamů použít pole bool hodnot
- Je třeba rozmyslet jak rozumně uložit seznam sousedů
    - vzpomeňme si na CSR (nebo CSC) formát pro uložení řídkých matic
        - jedno pole `edges_sousede` o délce `počtu hran` udávalo indexy sousedů
        - druhé pole `edges_index_sousedu` o délce `počtu vrcholů + 1` udávalo které indexy v prvním poli patří ke kterému vrcholu
    - toto můžeme vyrobit i ručně v Numpy bez nutnosti SciPy.sparse a to tak, že si:
        - nasčítáme počet odchozích hran z každého vrcholu a sesumujeme - vytvoříme `edges_index_sousedu`
            - pro pohodlnost pozdějšího indexování si uděláme pole o délce `počet vrcholů + 1` a na indexu `0` bude 0
            - pak provedeme kulmulaivní sumu
        - vytvoříme pole `edges_sousede` o délce `počet hran x 2` a do něj na správné indexy vložíme vrcholy sousedů
            - pomocí kopie pole `edges_index_sousedu` postupně zapíšeme pro každou hranu indexy vrcholů souseda, pomocí inkrementace v `edges_index_sousedu` si držíme aktuální polohu v poli `edges_sousede`

Budeme chtít také vstup jako Numpy:

In [ ]:
E_np = np.array(E)
E_np.shape

In [ ]:
def reachable_in_n_steps_np(edges_in, n):
    n_vertices = np.max(edges_in) + 1  # počet vrcholů

    # počet odchozích hran (kumulativně) pro každý vrchol (první záznam fixně 0)
    edges_index_sousedu = np.zeros(n_vertices + 1, dtype=np.int32)  
    # neprve nasčítáme počty hran
    for i in range(edges_in.shape[0]):
        edges_index_sousedu[edges_in[i, 0] + 1] += 1
        edges_index_sousedu[edges_in[i, 1] + 1] += 1
    # pak provedeme kumulativní součet
    for i in range(1, n_vertices + 1):
        edges_index_sousedu[i] += edges_index_sousedu[i - 1]

    # indexy sousedních vrcholů pro každý vrchol (tvar dle CSR/CSC formátu)
    edges_sousede = np.zeros(edges_in.size, dtype=np.int32)
    edges_tmp_index = edges_index_sousedu.copy() # kopie pro držení indexů k zápisu
    for i in range(edges_in.shape[0]):  # pro všechny hrany zapiš vrcholy souseda
        edges_sousede[edges_tmp_index[ edges_in[i, 0]]] = edges_in[i, 1]
        edges_tmp_index[ edges_in[i, 0]] += 1
        edges_sousede[edges_tmp_index[edges_in[i, 1]]] =  edges_in[i, 0]
        edges_tmp_index[edges_in[i, 1]] += 1

    reachable = np.zeros(n_vertices, dtype=np.bool_)  # maska dosažitelných vrcholů
    reachable[0] = True  # začínáme ve vrcholu 0
    newly_reachable = reachable.copy()  # vrcholy, které jsme právě přidali

    for _ in range(n):
        next_reachable = np.zeros(n_vertices, dtype=np.bool_)
        for v in np.where(newly_reachable)[0]:  # cyklus jen přes indexy nově dosažitelných vrcholů
            # přidáme všechny sousedy dle edges_sousede
            next_reachable[edges_sousede[edges_index_sousedu[v]:edges_index_sousedu[v + 1]]] = True

        # pouze nově přidané vrcholy = přidané teď a zároveň dosud nedosažitelné
        newly_reachable = np.logical_and(next_reachable, np.logical_not(reachable))

        # všechny dosažitelné vrcholy = dosud dosažitelné nebo nově dosažitelné
        reachable = np.logical_or(reachable, next_reachable)

        if not np.any(newly_reachable):
            break

    return np.where(reachable)[0]

In [ ]:
%timeit res3 = reachable_in_n_steps_np(E_np, 20)

In [ ]:
res3 = reachable_in_n_steps_np(E_np, 20)
np.allclose(np.array(res1), np.array(res3))

**Tak to jsme vlastně moc nevylepšili.**

Podíváme se kde se tráví většina času.

In [ ]:
%lprun -f reachable_in_n_steps_np reachable_in_n_steps_np(E_np, 20)

Zkusíme to zakompilovat, uvidíme jestli to pomůže.

### Implementační optimalizace II - Numba
- zkusíme co lze jednoduše zakompilovat

#### NumPy verze

In [ ]:
from numba import jit
import numpy as np

reachable_in_n_steps_np_numba = jit(reachable_in_n_steps_np, nopython=True)


In [ ]:
%time res4 = reachable_in_n_steps_np_numba(E_np, 20)

In [ ]:
%timeit res4 = reachable_in_n_steps_np_numba(E_np, 20)

To už je obrovské zrychlení!

Zkontrolujeme, že počítáme stále to samé:

In [ ]:
res4 = reachable_in_n_steps_np_numba(E_np, 20)
np.allclose(np.array(res1), np.array(res4))

#### Python verze (s listy a sety)
Možná by nás zajímalo jak by dopadla Numba pro:

`reachable_in_n_steps_v2_numba = jit(reachable_in_n_steps_v2, nopython=True)`


In [ ]:
reachable_in_n_steps_v2_numba = jit(reachable_in_n_steps_v2, nopython=True)


In [ ]:
%time _ = reachable_in_n_steps_v2_numba(E, 20)

In [ ]:
%timeit _ = reachable_in_n_steps_v2_numba(E, 20)

V případě `v3` Numba nelze přímočaře použít, neboť Numba podporuje pouze homogenní listy a sety. A my potřebujeme pro náš formát vyrobit list listů...

In [ ]:
# reachable_in_n_steps_v3_numba = jit(reachable_in_n_steps_v3, nopython=True)
# _ = reachable_in_n_steps_v3_numba(E, 20)

# Optimalizace III - Jiný přístup k úloze - použití matice sousednosti
- Pro zjištění hran, do kterých se lze dostat můžeme použít matici sousednosti
    - je to matice `n x n` kde na pozicích `[u, v] a [v, u]` je `True` pokud je hrana mezi vrcholy `u` a `v`
- Pokud mám vektor `reachable` složený z bool hodnot, kde `True` znamená, že vrchol je dosažitelný, tak mohu jednoduchým násobením matice sousednosti zjistit, které vrcholy mohu dosáhnout v dalším kroku
- Díky tomu, že je matice symetrická, a násobíme kumulativně, zachováváme ve vektoru `reachable` vždy i informaci o vrcholech, které jsme dosáhli v předchozích krocích

In [ ]:
from scipy.sparse import csr_matrix

def reachable_in_n_steps_scipy(edges, n):
    n_vertices = np.max(edges) + 1  # počet vrcholů

    # vytvoření matice sousednosti
    idx_row = np.concatenate((edges[:,0], edges[:,1]))
    idx_col = np.concatenate((edges[:,1], edges[:,0]))
    values = np.ones((len(idx_row)), dtype=np.bool_)
    adjacence_csc = csr_matrix((values, (idx_row, idx_col)), 
                               shape=(n_vertices, n_vertices), dtype=np.bool_)

    reachable = np.zeros((n_vertices), dtype=np.bool_)  # maska dosažitelných vrcholů
    reachable[0] = True
    for _ in range(n):  # počet kroků
        reachable_new = adjacence_csc.dot(reachable)  # nově dosažitelné vrcholy pomocí násobení matice
        if np.all(reachable == reachable_new):
            break
        reachable = reachable_new

    return np.where(reachable)[0]  # získání indexů dosažitelných vrcholů z masky

In [ ]:
%timeit res5 = reachable_in_n_steps_scipy(E_np, 20)

To celkem újde.

Zkontrolujeme, že počítáme stále to samé:

In [ ]:
res5 = reachable_in_n_steps_scipy(E_np, 20)
np.allclose(np.array(res1), res5)

In [ ]:
# profilovani
%lprun -f reachable_in_n_steps_scipy reachable_in_n_steps_scipy(E_np, 20)

## Benchmarkování nejlepších variant

Pro větší úlohy budeme chtít i rychlejší generování grafů.

In [ ]:
import numba
vygeneruj_graf = numba.jit(vygeneruj_graf)


### Závislost výpočetního času na rostoucím počtu kroků `n`
- Vynecháme nejpomalejší varianty `v1` a `v2` před kompilací Numbou.

In [ ]:
import matplotlib.pyplot as plt
import time

num_vert = 50000
V, E = vygeneruj_graf(num_vert)
E_np = np.array(E)

n_list = [2**i for i in range(0, 9)]
times_v2_numba = []  # reachable_in_n_steps_v2_numba
times_v3 = []  # reachable_in_n_steps_v3
times_np = [] # reachable_in_n_steps_np
times_np_numba = [] # reachable_in_n_steps_np_numba
times_scipy = [] # reachable_in_n_steps_scipy

for n in n_list:
    start = time.time()
    res = reachable_in_n_steps_v2_numba(E, n)
    end = time.time()
    times_v2_numba.append(end - start)

    start = time.time()
    res = reachable_in_n_steps_v3(E, n)
    end = time.time()
    times_v3.append(end - start)
    
    start = time.time()
    res = reachable_in_n_steps_np(E_np, n)
    end = time.time()
    times_np.append(end - start)
    
    start = time.time()
    res = reachable_in_n_steps_np_numba(E_np, n)
    end = time.time()
    times_np_numba.append(end - start)
    
    start = time.time()
    res = reachable_in_n_steps_scipy(E_np, n)
    end = time.time()
    times_scipy.append(end - start)

    print(n, times_v2_numba[-1], times_np[-1], times_np_numba[-1], times_scipy[-1])

# plot logaritmic scale on y axis

plt.figure(figsize=(10,10))
plt.loglog(n_list, times_v2_numba, label='v2 Numba')
plt.loglog(n_list, times_v3, label='v3')
plt.loglog(n_list, times_np, label='np')
plt.loglog(n_list, times_np_numba, label='np numba')
plt.loglog(n_list, times_scipy, label='scipy')

plt.xlabel('Number of steps')
plt.ylabel('Time [s]')
plt.title(f"Závislost výpočetního času na počtu kroků pro velikost grafu {num_vert}.")
plt.grid()
plt.legend()

Jen ty nejrychlejí a pro větší počet kroků:

In [ ]:
import matplotlib.pyplot as plt
import time

num_vert = 50000
V, E = vygeneruj_graf(num_vert,1)
E_np = np.array(E)

n_list = [2**i for i in range(0, 16)]
times_v3 = []  # reachable_in_n_steps_v3
times_np_numba = []  # reachable_in_n_steps_np_numba
times_scipy = []  # reachable_in_n_steps_scipy

for n in n_list:
    start = time.time()
    res = reachable_in_n_steps_v3(E, n)
    end = time.time()
    times_v3.append(end - start)

    start = time.time()
    res = reachable_in_n_steps_np_numba(E_np, n)
    end = time.time()
    times_np_numba.append(end - start)

    start = time.time()
    res = reachable_in_n_steps_scipy(E_np, n)
    end = time.time()
    times_scipy.append(end - start)

    print(n, times_v3[-1], times_np_numba[-1], times_scipy[-1])

# plot logaritmic scale on y axis

plt.figure(figsize=(10, 10))
plt.loglog(n_list, times_v3, label='v3')
plt.loglog(n_list, times_np_numba, label='np numba')
plt.loglog(n_list, times_scipy, label='scipy')

plt.xlabel('Number of steps')
plt.ylabel('Time [s]')
plt.title(f"Závislost výpočetního času na počtu kroků pro velikost grafu {num_vert}.")
plt.grid()
plt.legend()

### Závislost výpočetního času na rostoucím počtu prcholů/hran

In [ ]:
import time
import matplotlib.pyplot as plt


n = 10  # počet kroků
num_vert_list = [2**i for i in range(1, 14)]
times_v2_numba = [] # reachable_in_n_steps_v2_numba
times_v3 = []  # reachable_in_n_steps_v3
times_np = [] # reachable_in_n_steps_np
times_np_numba = [] # reachable_in_n_steps_numba
times_scipy = [] # reachable_in_n_steps_scipy

for num_vert in num_vert_list:
    V, E = vygeneruj_graf(num_vert)
    E_np = np.array(E) 

    start = time.time()
    res = reachable_in_n_steps_v2_numba(E, n)
    end = time.time()
    times_v2_numba.append(end - start)

    start = time.time()
    res = reachable_in_n_steps_v3(E, n)
    end = time.time()
    times_v3.append(end - start)
    
    start = time.time()
    res = reachable_in_n_steps_np(E_np, n)
    end = time.time()
    times_np.append(end - start)
    
    start = time.time()
    res = reachable_in_n_steps_np_numba(E_np, n)
    end = time.time()
    times_np_numba.append(end - start)
    
    start = time.time()
    res = reachable_in_n_steps_scipy(E_np, n)
    end = time.time()
    times_scipy.append(end - start)

    print(num_vert, times_v2_numba[-1], times_np[-1], times_np_numba[-1], times_scipy[-1])

# plot logaritmic scale on y axis

plt.figure(figsize=(10,10))
plt.loglog(num_vert_list, times_v2_numba, label='v2 numba')
plt.loglog(num_vert_list, times_v3, label='v3')
plt.loglog(num_vert_list, times_np, label='np')
plt.loglog(num_vert_list, times_np_numba, label='np numba')
plt.loglog(num_vert_list, times_scipy, label='scipy')

plt.xlabel('Graph size')
plt.ylabel('Time [s]')
plt.title(f"Závislost výpočetního času na velikosti grafu pro počet kroků {n}.")
plt.grid()

plt.legend()

Bez `v2` a `np` a pro větší grafy.

In [ ]:
import time
import matplotlib.pyplot as plt


n = 10  # počet kroků
num_vert_list = [2**i for i in range(1, 18)]
times_v3 = []  # reachable_in_n_steps_v3
times_np_numba = []  # reachable_in_n_steps_numba
times_scipy = []  # reachable_in_n_steps_scipy

for num_vert in num_vert_list:
    V, E = vygeneruj_graf(num_vert)
    E_np = np.array(E)

    start = time.time()
    res = reachable_in_n_steps_v3(E, n)
    end = time.time()
    times_v3.append(end - start)

    start = time.time()
    res = reachable_in_n_steps_np_numba(E_np, n)
    end = time.time()
    times_np_numba.append(end - start)

    start = time.time()
    res = reachable_in_n_steps_scipy(E_np, n)
    end = time.time()
    times_scipy.append(end - start)

    print(num_vert, times_np[-1], times_np_numba[-1], times_scipy[-1])

# plot logaritmic scale on y axis

plt.figure(figsize=(10, 10))
plt.loglog(num_vert_list, times_v3, label='v3')
plt.loglog(num_vert_list, times_np_numba, label='np numba')
plt.loglog(num_vert_list, times_scipy, label='scipy')

plt.xlabel('Graph size')
plt.ylabel('Time [s]')
plt.title(f"Závislost výpočetního času na velikosti grafu pro počet kroků {n}.")
plt.grid()

plt.legend()